# Reading .h5 keys

In [2]:
import h5py,sys,os
import numpy as np
# Function to print the keys of an HDF5 file

if __name__ == "__main__":
	filepath="/home/hep/lr1424/1P1Qm/flat_train/TTBar+ZJets_flat.h5"
	f= h5py.File(filepath,'r')
	print(f.keys())


	# Print unique entries for all non-data keys
	non_data_keys = [
		'jetConstituentNames', 'jetConstituentsExtraNames',
		'jetFeatureNames', 
		'subjet_feature_names', 'subjet_labels', 'truth_labels'
	]

	for key in non_data_keys:
		data = f[key][:]
		unique_entries = set(data.flat) if hasattr(data, 'flat') else set(data)
		print(f"Unique entries in '{key}':", unique_entries)
  
# Get the shape of 'jetConstituentsList' dataset
if 'jetConstituentsList' in f:
	shape = f['jetConstituentsList'].shape
	print(f"Shape of 'jetConstituentsList': {shape}")
else:
	print("'jetConstituentsList' not found in file.")
 
shape1 = f['jetConstituentNames'].shape
print(f"Shape of 'jetConstituentNames': {shape1}")
print([x for x in f['jetConstituentNames']])

shape2 = f['truth_labels'].shape
print(f"Shape of 'truth_labels': {shape2}")
print([x for x in f['truth_labels']][:10])  # Print first 10 truth labels
# Print the first 3 jets and first 5 particles per jet with their pT, eta, phi
jets = f['jetConstituentsList'][:3, :5, :3]  # shape: (3, 5, 3)
for jet_idx, jet in enumerate(jets):
	print(f"Jet {jet_idx}:")
	for part_idx, (eta, phi, pt) in enumerate(jet):
		eta_deg = np.degrees(eta)
		phi_deg = np.degrees(phi)
		print(f"  Particle {part_idx}: eta={eta_deg:.3f}°, phi={phi_deg:.3f}°, pT={pt:.3f}")
	print()
 
# Find max and min pt, eta, and phi from the whole file
jet_constituents = f['jetConstituentsList'][:]  # shape: (N_jets, N_particles, 3)
eta_all = jet_constituents[..., 0]
phi_all = jet_constituents[..., 1]
pt_all = jet_constituents[..., 2]

print(f"pt: min={pt_all.min():.3f}, max={pt_all.max():.3f}")
print(f"eta: min={eta_all.min():.3f}, max={eta_all.max():.3f}")
print(f"phi: min={phi_all.min():.3f}, max={phi_all.max():.3f}")

<KeysViewHDF5 ['PFCand_subjet_idx', 'jetConstituentNames', 'jetConstituentsExtra', 'jetConstituentsExtraNames', 'jetConstituentsList', 'jetFeatureNames', 'jetFeatures', 'num_PFCands_subleading_jet', 'subjet_feature_names', 'subjet_features', 'subjet_labels', 'truth_labels']>
Unique entries in 'jetConstituentNames': {'pt', 'phi', 'eta'}
Unique entries in 'jetConstituentsExtraNames': {'part_isChargedHadron', 'part_dzval', 'part_d0err', 'part_isMuon', 'part_isElectron', 'part_charge', 'part_isPhoton', 'part_isNeutralHadron', 'part_dzerr', 'part_d0val'}
Unique entries in 'jetFeatureNames': {'jet_phi', 'jet_tau2', 'jet_pt', 'jet_energy', 'jet_tau1', 'jet_nparticles', 'jet_eta', 'jet_tau4', 'jet_tau3', 'jet_sdmass'}
Unique entries in 'subjet_feature_names': {'sj1Eta', 'sj2Pt', 'sj1Pt', 'sj2Eta', 'sj2Phi', 'sj1Phi', 'sj1E', 'sj2E'}
Unique entries in 'subjet_labels': {0.0, 1.0, -1.0}
Unique entries in 'truth_labels': {0.0, 1.0}
Shape of 'jetConstituentsList': (40000, 100, 3)
Shape of 'jetConst

# Observing Loss

In [1]:
import glob
import re

log_files = glob.glob("/home/hep/lr1424/1P1Qm_fork/output_logs/*.log")
pattern = re.compile(r"Epoch\s+(\d+)/(\d+)\s+-\s+Training Loss:\s+([\d.]+)\s+-\s+Validation Loss:\s+([\d.]+)")

for log_file in log_files:
    print(f"Processing {log_file}")
    with open(log_file, "r") as f:
        for line in f:
            match = pattern.search(line)
            if match:
                epoch, total_epochs, train_loss, val_loss = match.groups()
                print(f"\t Epoch {epoch}/{total_epochs} - Training Loss: {train_loss} - Validation Loss: {val_loss}")
    print()

Processing /home/hep/lr1424/1P1Qm_fork/output_logs/2000_jets.log
	 Epoch 1/10 - Training Loss: 0.6948 - Validation Loss: 0.6930
	 Epoch 2/10 - Training Loss: 0.6948 - Validation Loss: 0.6930

Processing /home/hep/lr1424/1P1Qm_fork/output_logs/100_jets_MSE.log
	 Epoch 1/10 - Training Loss: 0.2514 - Validation Loss: 0.2501
	 Epoch 2/10 - Training Loss: 0.2513 - Validation Loss: 0.2506
	 Epoch 3/10 - Training Loss: 0.2513 - Validation Loss: 0.2509
	 Epoch 4/10 - Training Loss: 0.2513 - Validation Loss: 0.2511
	 Epoch 5/10 - Training Loss: 0.2513 - Validation Loss: 0.2512
	 Epoch 6/10 - Training Loss: 0.2513 - Validation Loss: 0.2512
	 Epoch 7/10 - Training Loss: 0.2513 - Validation Loss: 0.2513
	 Epoch 8/10 - Training Loss: 0.2513 - Validation Loss: 0.2513
	 Epoch 9/10 - Training Loss: 0.2513 - Validation Loss: 0.2513
	 Epoch 10/10 - Training Loss: 0.2513 - Validation Loss: 0.2513

Processing /home/hep/lr1424/1P1Qm_fork/output_logs/100_jets_BCE.log
	 Epoch 1/10 - Training Loss: 0.6959 - V

In [1]:
import strawberryfields as sf
import tensorflow as tf
sf.about()
print("Num PUs Available: ", len(tf.config.list_physical_devices()))
print(tf.config.list_physical_devices())
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA (GPU) support.")
else:
    print("TensorFlow is CPU-only (no GPU support).")

2025-07-02 18:17:03.259699: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-02 18:17:03.261904: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-02 18:17:03.302229: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-02 18:17:03.303193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-02 18:17:04.200669: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co


Strawberry Fields: a Python library for continuous-variable quantum circuits.
Copyright 2018-2020 Xanadu Quantum Technologies Inc.

Python version:            3.8.20
Platform info:             Linux-4.18.0-477.27.1.el8_8.x86_64-x86_64-with-glibc2.17
Installation path:         /rds/general/user/lr1424/home/miniconda3/envs/qml-env/lib/python3.8/site-packages/strawberryfields
Strawberry Fields version: 0.23.0
Numpy version:             1.24.3
Scipy version:             1.10.1
SymPy version:             1.13.3
NetworkX version:          3.1
The Walrus version:        0.21.0
Blackbird version:         0.5.0
XCC version:               0.3.1
TensorFlow version:        2.13.1
Num PUs Available:  1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
TensorFlow is built with CUDA (GPU) support.


In [2]:
import importlib.metadata
print(importlib.metadata.version('typing_extensions'))

4.5.0


# Heavily Simplified Version - Testing for non-zero gradients

### Working basic example w/ demo circuit from [here](https://strawberryfields.ai/photonics/demos/run_tutorial_machine_learning.html)

In [3]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, Sgate, BSgate, CXgate
import tensorflow as tf, numpy as np, h5py, os, random

# ----------  hyper-params ----------
CUT_OFF     = 7                 # fock cutoff dim
WIRES       = 1
LAYERS      = 1
STEPS       = 50
LR          = 0.01
MAX_JETS    = 200               # keep it tiny for the demo
DATA_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_train/TTBar+ZJets_flat.h5"
# -----------------------------------

# ----------  load a small dataset ----------
with h5py.File(DATA_DIR, "r") as f:
    X = f["jetConstituentsList"][:MAX_JETS, :WIRES, :]     # (N,4,3)
    y = f["truth_labels"][:MAX_JETS].astype(np.float32)     # (N,)

jets   = tf.convert_to_tensor(X, tf.float32)
labels = tf.convert_to_tensor(y, tf.float32)

prog = sf.Program(WIRES)

alpha, phi = prog.params("alpha", "phi")

with prog.context as q:
    Dgate(alpha, phi) | q[0]

tf_alpha = tf.Variable(0.1)
tf_phi = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(LR)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mse = tf.keras.losses.MeanSquaredError()

eng = sf.Engine("tf", backend_options={"cutoff_dim": CUT_OFF})

# -------- training loop ----------
for step in range(STEPS):
    i = random.randrange(MAX_JETS)
    jet, label = jets[i], labels[i]

    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        state = eng.run(prog, args={"alpha": tf_alpha, "phi": tf_phi}).state
        # print(state)
        # print(state.mean_photon(0))
        # print(tf.shape(state.mean_photon(0)),  "---", tf.shape(label))
        mean_photon = state.mean_photon(0)[0]          # discard the variance
        y_pred      = tf.expand_dims(mean_photon, 0)   # shape (1,)
        y_true      = tf.expand_dims(label, 0)         # shape (1,)

        loss = mse(y_true, y_pred)

    vars_ = [tf_alpha, tf_phi] 
    grads = tape.gradient(loss, vars_)
    opt.apply_gradients(zip(grads, vars_))

    if step % 5 == 0:
        gnorms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
        print(f"step {step:4d}  loss={loss.numpy():.4f}  "
              f"gnorms={['%.1e'%n for n in gnorms[:6]]}")


step    0  loss=0.9801  gnorms=['4.0e-01', '2.2e-10']
step    5  loss=0.0005  gnorms=['1.3e-02', '1.3e-11']
step   10  loss=0.0013  gnorms=['2.7e-02', '0.0e+00']
step   15  loss=0.8988  gnorms=['8.6e-01', '2.6e-10']
step   20  loss=0.8630  gnorms=['9.9e-01', '2.8e-09']
step   25  loss=0.0091  gnorms=['1.2e-01', '4.5e-13']
step   30  loss=0.7789  gnorms=['1.2e+00', '1.9e-09']
step   35  loss=0.0239  gnorms=['2.4e-01', '3.6e-10']
step   40  loss=0.6565  gnorms=['1.4e+00', '4.7e-10']
step   45  loss=0.0560  gnorms=['4.6e-01', '6.2e-11']


### Working with simplified version of my circuit (after scaling pT down)

In [4]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, Sgate, BSgate, CXgate
import tensorflow as tf, numpy as np, h5py, os, random

# ----------  hyper-params ----------
CUT_OFF     = 7                 # fock cutoff dim
WIRES       = 1
LAYERS      = 1
STEPS       = 50
LR          = 0.01
MAX_JETS    = 200               # keep it tiny for the demo
DATA_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_train/TTBar+ZJets_flat.h5"
# -----------------------------------

# ----------  load a small dataset ----------
with h5py.File(DATA_DIR, "r") as f:
    X = f["jetConstituentsList"][:MAX_JETS, :WIRES, :]     # (N,4,3)
    y = f["truth_labels"][:MAX_JETS].astype(np.float32)     # (N,)

jets   = tf.convert_to_tensor(X, tf.float32)
labels = tf.convert_to_tensor(y, tf.float32)

# -------- symbolic circuit ----------
prog = sf.Program(WIRES)
s_scale = prog.params("s_scale")

eta = prog.params("eta")
phi = prog.params("phi")
pt  = prog.params("pt")

with prog.context as q:
    scale = 10.0 / (1 + sf.math.exp(-s_scale)) + 0.01
    Sgate(eta, pt*phi/2) | q[0]
    Dgate(scale*pt, eta)    | q[0]
    # CXgate(1.0) | (q[0], q[1])

# # -------- one tf.Variable per placeholder  ############## FIX ##############
rnd = tf.random_uniform_initializer(-0.1, 0.1)
tf_s_scale = tf.Variable(rnd(()))

def make_args(jet):
    d = {"s_scale": tf_s_scale}    
    pt, eta, phi = jet[0, 2], jet[0, 0], jet[0, 1]
    # Scale pt to [0, 1] via Aritra's case_reader.py
    # Use dictionaries for feature limits
    assumed_limits = {
        'pt': [1e-4, 3000.0],    # assumed limits for pt
        'eta': [-0.8, 0.8],      # example limits for eta
        'phi': [-0.8, 0.8],  # example limits for phi
    }
    feature_limits = {
        'pt': [0.0, 1.0],        # limits for the feature space
        'eta': [-np.pi, np.pi],
        'phi': [-np.pi, np.pi],
    }
    scaled_pt = ((pt - assumed_limits["pt"][0]) / (assumed_limits["pt"][1] - assumed_limits["pt"][0])) * (feature_limits["pt"][1] - feature_limits["pt"][0]) + feature_limits["pt"][0]
    scaled_eta = ((eta - assumed_limits["eta"][0]) / (assumed_limits["eta"][1] - assumed_limits["eta"][0])) * (feature_limits["eta"][1] - feature_limits["eta"][0]) + feature_limits["eta"][0]
    scaled_phi = ((phi - assumed_limits["phi"][0]) / (assumed_limits["phi"][1] - assumed_limits["phi"][0])) * (feature_limits["phi"][1] - feature_limits["phi"][0]) + feature_limits["phi"][0]
    # print("pt =", pt, "scaled_pt =",scaled_pt)
    pt = scaled_pt
    eta = scaled_eta
    phi = scaled_phi

    d[f"eta"] = eta
    d[f"phi"] = phi
    d[f"pt"]  = pt

    return d

opt = tf.keras.optimizers.Adam(LR)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

eng = sf.Engine("tf", backend_options={"cutoff_dim": CUT_OFF})

# -------- training loop ----------
for step in range(STEPS):
    i = random.randrange(MAX_JETS)
    jet, label = jets[i], labels[i]

    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        state = eng.run(prog, args=make_args(jet)).state
        mean_photon = state.mean_photon(0)[0]          # discard the variance
        y_pred      = tf.expand_dims(mean_photon, 0)   # shape (1,)
        y_true      = tf.expand_dims(label, 0)         # shape (1,)

        loss = bce(y_true, y_pred)

    vars_ = [tf_s_scale]
    grads = tape.gradient(loss, vars_)
    opt.apply_gradients(zip(grads, vars_))

    if step % 5 == 0:
        gnorms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
        print(f"step {step:4d}  loss={loss.numpy():.4f}  "
              f"gnorms={['%.1e'%n for n in gnorms[:6]]}")

step    0  loss=0.6722  gnorms=['2.1e-02']
step    5  loss=0.6993  gnorms=['4.6e-03']
step   10  loss=0.6862  gnorms=['2.4e-03']
step   15  loss=0.6442  gnorms=['3.7e-02']
step   20  loss=0.6141  gnorms=['1.2e-02']
step   25  loss=0.8035  gnorms=['1.1e-01']
step   30  loss=0.7192  gnorms=['2.6e-02']
step   35  loss=0.8194  gnorms=['1.3e-01']
step   40  loss=0.6726  gnorms=['5.5e-03']
step   45  loss=0.7056  gnorms=['8.1e-03']


### Seemingly working with full circuit ... except

In [5]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, Sgate, BSgate, CXgate
import tensorflow as tf, numpy as np, h5py, os, random

# ----------  hyper-params ----------
CUT_OFF     = 7                 # fock cutoff dim
WIRES       = 4
LAYERS      = 1
STEPS       = 50
LR          = 0.01
MAX_JETS    = 200               # keep it tiny for the demo
DATA_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_train/TTBar+ZJets_flat.h5"
# -----------------------------------

# ----------  load a small dataset ----------
with h5py.File(DATA_DIR, "r") as f:
    X = f["jetConstituentsList"][:MAX_JETS, :WIRES, :]     # (N,4,3)
    y = f["truth_labels"][:MAX_JETS].astype(np.float32)     # (N,)

jets   = tf.convert_to_tensor(X, tf.float32)
labels = tf.convert_to_tensor(y, tf.float32)

# -------- symbolic circuit ----------
prog = sf.Program(WIRES)
s_scale = prog.params("s_scale")
DM  = [prog.params(f"DM{w}") for w in range(WIRES)]
DP  = [prog.params(f"DP{w}") for w in range(WIRES)]
SM  = [prog.params(f"SM{w}") for w in range(WIRES)]
SP  = [prog.params(f"SP{w}") for w in range(WIRES)]
eta = [prog.params(f"eta{w}") for w in range(WIRES)]
phi = [prog.params(f"phi{w}") for w in range(WIRES)]
pt  = [prog.params(f"pt{w}")  for w in range(WIRES)]


with prog.context as q:
    scale = 10.0 / (1.0 + sf.math.exp(-s_scale)) + 0.01
    for w in range(WIRES):
        Sgate(eta[w], pt[w]*phi[w]/2) | q[w]
        Dgate(scale*pt[w], eta[w])    | q[w]
    for a,b in [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]:
        CXgate(1.0) | (q[a], q[b])
    for w in range(WIRES):
        Sgate(SM[w], SP[w]) | q[w]
        Dgate(DM[w], DP[w]) | q[w]

# # -------- one tf.Variable per placeholder 
rnd = tf.random_uniform_initializer(-0.1, 0.1)
tf_s_scale = tf.Variable(rnd(()))
tf_DM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_DP = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SP = [tf.Variable(rnd(())) for _ in range(WIRES)]

assumed_limits = {
    'pt':  [1e-4, 3000.0],
    'eta': [-0.8, 0.8],
    'phi': [-0.8, 0.8],
}
feature_limits = {
    'pt':  [0.0, 1.0],
    'eta': [-np.pi, np.pi],
    'phi': [-np.pi, np.pi],
}

def scale_feature(value, name):
    """Affine-map any feature from its assumed data range to the network range."""
    a_min, a_max = assumed_limits[name]
    f_min, f_max = feature_limits[name]
    return (value - a_min) / (a_max - a_min) * (f_max - f_min) + f_min

def make_args(jet):
    d = {"s_scale": tf_s_scale}
    for w in range(WIRES):
        d[f"DM{w}"] = tf_DM[w]
        d[f"DP{w}"] = tf_DP[w]
        d[f"SM{w}"] = tf_SM[w]
        d[f"SP{w}"] = tf_SP[w]

        d[f"eta{w}"] = scale_feature(jet[w, 0], "eta")
        d[f"phi{w}"] = scale_feature(jet[w, 1], "phi")
        d[f"pt{w}"]  = scale_feature(jet[w, 2], "pt")
    return d

opt = tf.keras.optimizers.Adam(LR)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

eng = sf.Engine("tf", backend_options={"cutoff_dim": CUT_OFF})

# -------- training loop ----------
for step in range(STEPS):
    i = random.randrange(MAX_JETS)
    jet, label = jets[i], labels[i]
    # print("pt0 =", jet[0, 2].numpy())      # add just after you sample `jet`
    
    # print("scale*pt =", tf_s_scale*pt)
    # print("pt*pt*phi/2 =", pt*pt*phi/2)


    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        state = eng.run(prog, args=make_args(jet)).state
        photons = tf.stack([state.mean_photon(m) for m in range(3)])
        logit   = tf.reduce_sum(photons)
        loss    = bce(tf.expand_dims(label,0), tf.expand_dims(logit,0))
        # mean_photon = state.mean_photon(0)[0]          # discard the variance
        # y_pred      = tf.expand_dims(mean_photon, 0)   # shape (1,)
        # y_true      = tf.expand_dims(label, 0)         # shape (1,)

        # loss = bce(y_true, y_pred)

    vars_ = [tf_s_scale, *tf_DM, *tf_DP, *tf_SM, *tf_SP]  
    grads = tape.gradient(loss, vars_)
    opt.apply_gradients(zip(grads, vars_))

    if step % 5 == 0:
        gnorms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
        print(f"step {step:4d}  loss={loss.numpy():.4f}  "
              f"gnorms={['%.1e'%n for n in gnorms[:6]]}")

step    0  loss=0.0568  gnorms=['2.3e-03', '2.6e-03', '8.9e-03', '6.7e-03', '1.0e-03', '8.2e-05']
step    5  loss=2.0657  gnorms=['4.4e-01', '3.3e-01', '4.5e-01', '3.8e-01', '1.1e-02', '2.2e-03']
step   10  loss=1.7634  gnorms=['3.0e-01', '7.6e-02', '4.1e-01', '5.1e-01', '1.3e-02', '2.4e-05']
step   15  loss=1.7323  gnorms=['4.1e-01', '3.5e-02', '3.8e-01', '5.4e-01', '2.1e-02', '1.2e-03']
step   20  loss=2.1158  gnorms=['3.6e-01', '4.4e-03', '5.6e-01', '2.3e-01', '3.1e-02', '6.5e-03']
step   25  loss=0.0654  gnorms=['4.6e-03', '1.9e-03', '4.1e-03', '1.5e-02', '3.3e-03', '4.5e-04']
step   30  loss=1.6088  gnorms=['2.1e-01', '1.2e-01', '2.3e-01', '2.3e-01', '3.1e-02', '3.5e-04']
step   35  loss=1.5967  gnorms=['2.9e-01', '1.6e-01', '4.1e-02', '9.6e-02', '3.4e-02', '1.0e-02']
step   40  loss=0.2684  gnorms=['4.7e-02', '2.5e-02', '3.8e-02', '5.8e-02', '1.2e-02', '4.7e-03']
step   45  loss=1.3017  gnorms=['2.6e-01', '7.4e-02', '8.4e-02', '1.6e-01', '2.6e-02', '9.4e-04']


### Same as above (with additional debugging); keeping above to show it worked at least once.

- Only works first time, then get the error:
```
WARNING:tensorflow:Gradients do not exist for variables ['Variable:0'] when minimizing the loss.
step    0  loss=0.2500  gnorms=['0.0e+00', '8.2e-03', '1.2e-03', '1.7e-02', '2.9e-03', '2.5e-05']
```

In [ ]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, Sgate, BSgate, CXgate
import tensorflow as tf, numpy as np, h5py, os, random

# ----------  hyper-params ----------
CUT_OFF     = 7                 # fock cutoff dim
WIRES       = 4
LAYERS      = 1
STEPS       = 50
LR          = 0.01
MAX_JETS    = 200               # keep it tiny for the demo
DATA_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_train/TTBar+ZJets_flat.h5"
# -----------------------------------

# ----------  load a small dataset ----------
with h5py.File(DATA_DIR, "r") as f:
    X = f["jetConstituentsList"][:MAX_JETS, :WIRES, :]     # (N,4,3)
    y = f["truth_labels"][:MAX_JETS].astype(np.float32)     # (N,)

jets   = tf.convert_to_tensor(X, tf.float32)
labels = tf.convert_to_tensor(y, tf.float32)

# -------- symbolic circuit ----------
prog = sf.Program(WIRES)
s_scale = prog.params("s_scale")
DM  = [prog.params(f"DM{w}") for w in range(WIRES)]
DP  = [prog.params(f"DP{w}") for w in range(WIRES)]
SM  = [prog.params(f"SM{w}") for w in range(WIRES)]
SP  = [prog.params(f"SP{w}") for w in range(WIRES)]
eta = [prog.params(f"eta{w}") for w in range(WIRES)]
phi = [prog.params(f"phi{w}") for w in range(WIRES)]
pt  = [prog.params(f"pt{w}")  for w in range(WIRES)]


with prog.context as q:
    scale = 10.0 / (1.0 + sf.math.exp(-s_scale)) + 0.01
    for w in range(WIRES):
        Sgate(eta[w], pt[w]*phi[w]/2) | q[w]
        Dgate(scale*pt[w], eta[w])    | q[w]
    for a,b in [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]:
        CXgate(1.0) | (q[a], q[b])
    for w in range(WIRES):
        Sgate(SM[w], SP[w]) | q[w]
        Dgate(DM[w], DP[w]) | q[w]

# # -------- one tf.Variable per placeholder 
rnd = tf.random_uniform_initializer(-0.1, 0.1)
tf_s_scale = tf.Variable(rnd(()))
# print("tf_s_scale =", tf_s_scale.numpy())
tf_DM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_DP = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SP = [tf.Variable(rnd(())) for _ in range(WIRES)]

assumed_limits = {
    'pt':  [1e-4, 3000.0],
    'eta': [-0.8, 0.8],
    'phi': [-0.8, 0.8],
}
feature_limits = {
    'pt':  [0.0, 1.0],
    'eta': [-np.pi, np.pi],
    'phi': [-np.pi, np.pi],
}

# EPS = 1e-3          

def scale_feature(value, name):
    """Affine-map any feature from its assumed data range to the network range."""
    a_min, a_max = assumed_limits[name]
    f_min, f_max = feature_limits[name]
    out = (value - a_min) / (a_max - a_min) * (f_max - f_min) + f_min
    # if name == "pt":
    #     out = tf.maximum(out, EPS)
    return out

def make_args(jet):
    d = {"s_scale": tf_s_scale}
    for w in range(WIRES):
        d[f"DM{w}"] = tf_DM[w]
        d[f"DP{w}"] = tf_DP[w]
        d[f"SM{w}"] = tf_SM[w]
        d[f"SP{w}"] = tf_SP[w]

        d[f"eta{w}"] = scale_feature(jet[w, 0], "eta")
        d[f"phi{w}"] = scale_feature(jet[w, 1], "phi")
        d[f"pt{w}"]  = scale_feature(jet[w, 2], "pt")
        # print("Dgate w/", tf_s_scale.numpy()*d[f"pt{w}"].numpy())
    return d

opt = tf.keras.optimizers.Adam(LR)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)

eng = sf.Engine("tf", backend_options={"cutoff_dim": CUT_OFF})

# -------- training loop ----------
for step in range(STEPS):
    i = random.randrange(MAX_JETS)
    jet, label = jets[i], labels[i]
    if tf.reduce_all(jet[..., 2] == 0.0):
        continue 
    # print("pt0 =", jet[0, 2].numpy())      # add just after you sample `jet`
    
    # print("scale*pt =", tf_s_scale*pt)
    # print("pt*pt*phi/2 =", pt*pt*phi/2)


    if eng.run_progs:
        eng.reset()

    with tf.GradientTape() as tape:
        tape.watch(tf_s_scale)
        state = eng.run(prog, args=make_args(jet)).state
        photons = tf.stack([state.mean_photon(m) for m in range(3)])
        logit   = tf.reduce_sum(photons)
        loss    = bce(tf.expand_dims(label,0), tf.expand_dims(logit,0))
        # mean_photon = state.mean_photon(0)[0]          # discard the variance
        # y_pred      = tf.expand_dims(mean_photon, 0)   # shape (1,)
        # y_true      = tf.expand_dims(label, 0)         # shape (1,)

        # loss = bce(y_true, y_pred)

    vars_ = [tf_s_scale, *tf_DM, *tf_DP, *tf_SM, *tf_SP]  
    grads = tape.gradient(loss, vars_)
    if grads[0] is None:
        print("⚠️  grad None; pT’s were", jet[:, 2].numpy())
    opt.apply_gradients(zip(grads, vars_))

    if step % 5 == 0:
        gnorms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
        print(f"step {step:4d}  loss={loss.numpy():.4f}  "
              f"gnorms={['%.1e'%n for n in gnorms[:6]]}")

⚠️  grad None; pT’s were [240.02118  125.55173   78.10409   55.470455]
step    0  loss=2.3560  gnorms=['0.0e+00', '3.2e-01', '3.2e-01', '5.5e-01', '1.6e-02', '4.1e-03']
⚠️  grad None; pT’s were [713.04114   23.53077   23.516682  18.784554]
⚠️  grad None; pT’s were [496.59155  137.78369  114.573     50.280598]
⚠️  grad None; pT’s were [68.94876  50.186806 49.476826 49.182854]
⚠️  grad None; pT’s were [95.76376  59.773476 38.720997 38.42305 ]
⚠️  grad None; pT’s were [80.27804  54.4434   44.192066 29.433529]
step    5  loss=2.2157  gnorms=['0.0e+00', '1.1e-01', '3.3e-03', '2.4e-02', '7.9e-02', '2.0e-03']
⚠️  grad None; pT’s were [307.49683  145.66698   95.09012   37.446888]
⚠️  grad None; pT’s were [88.96303  77.79551  71.32266  37.119316]
⚠️  grad None; pT’s were [176.4177   130.43      79.05531   50.509792]
⚠️  grad None; pT’s were [192.16554   76.00975   68.331215  55.938663]
⚠️  grad None; pT’s were [135.75758   88.16328   74.204605  67.8978  ]
step   10  loss=1.7244  gnorms=['0.0e+0

: 

### Add test and Val

In [ ]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, Sgate, BSgate, CXgate
import tensorflow as tf, numpy as np, h5py, os, random
from sklearn.metrics import roc_auc_score

# ----------  hyper-params ----------
CUT_OFF     = 7                 # fock cutoff dim
WIRES       = 4
LAYERS      = 1
STEPS       = 50
LR          = 0.01
MAX_JETS    = 200               # keep it tiny for the demo
DATA_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_train/TTBar+ZJets_flat.h5"
VAL_DIR     = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_val/TTBar+ZJets_flat.h5"
TEST_DIR    = "/rds/general/user/lr1424/home/1P1Qm_SF/flat_test/TTBar+ZJets_flat.h5"
# -----------------------------------

# ----------  load datasets ----------
def load_data(path, max_jets=MAX_JETS):
    with h5py.File(path, "r") as f:
        X = f["jetConstituentsList"][:max_jets, :WIRES, :]     # (N,4,3)
        y = f["truth_labels"][:max_jets].astype(np.float32)     # (N,)
    return tf.convert_to_tensor(X, tf.float32), tf.convert_to_tensor(y, tf.float32)

jets, labels = load_data(DATA_DIR)
jets_val, labels_val = load_data(VAL_DIR)
jets_test, labels_test = load_data(TEST_DIR)

# -------- symbolic circuit ----------
prog = sf.Program(WIRES)
s_scale = prog.params("s_scale")
DM  = [prog.params(f"DM{w}") for w in range(WIRES)]
DP  = [prog.params(f"DP{w}") for w in range(WIRES)]
SM  = [prog.params(f"SM{w}") for w in range(WIRES)]
SP  = [prog.params(f"SP{w}") for w in range(WIRES)]
eta = [prog.params(f"eta{w}") for w in range(WIRES)]
phi = [prog.params(f"phi{w}") for w in range(WIRES)]
pt  = [prog.params(f"pt{w}")  for w in range(WIRES)]

with prog.context as q:
    scale = 10.0 / (1.0 + sf.math.exp(-s_scale)) + 0.01
    for w in range(WIRES):
        Sgate(eta[w], pt[w]*phi[w]/2) | q[w]
        Dgate(scale*pt[w], eta[w])    | q[w]
    for a,b in [(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)]:
        CXgate(1.0) | (q[a], q[b])
    for w in range(WIRES):
        Sgate(SM[w], SP[w]) | q[w]
        Dgate(DM[w], DP[w]) | q[w]

rnd = tf.random_uniform_initializer(-0.1, 0.1)
tf_s_scale = tf.Variable(rnd(()))
tf_DM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_DP = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SM = [tf.Variable(rnd(())) for _ in range(WIRES)]
tf_SP = [tf.Variable(rnd(())) for _ in range(WIRES)]

assumed_limits = {
    'pt':  [1e-4, 3000.0],
    'eta': [-0.8, 0.8],
    'phi': [-0.8, 0.8],
}
feature_limits = {
    'pt':  [0.0, 1.0],
    'eta': [-np.pi, np.pi],
    'phi': [-np.pi, np.pi],
}

def scale_feature(value, name):
    a_min, a_max = assumed_limits[name]
    f_min, f_max = feature_limits[name]
    return (value - a_min) / (a_max - a_min) * (f_max - f_min) + f_min

def make_args(jet):
    d = {"s_scale": tf_s_scale}
    for w in range(WIRES):
        d[f"DM{w}"] = tf_DM[w]
        d[f"DP{w}"] = tf_DP[w]
        d[f"SM{w}"] = tf_SM[w]
        d[f"SP{w}"] = tf_SP[w]
        d[f"eta{w}"] = scale_feature(jet[w, 0], "eta")
        d[f"phi{w}"] = scale_feature(jet[w, 1], "phi")
        d[f"pt{w}"]  = scale_feature(jet[w, 2], "pt")
    return d

opt = tf.keras.optimizers.Adam(LR)
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
eng = sf.Engine("tf", backend_options={"cutoff_dim": CUT_OFF})

# -------- training loop ----------
for step in range(STEPS):
    i = random.randrange(MAX_JETS)
    jet, label = jets[i], labels[i]
    if eng.run_progs:
        eng.reset()
    with tf.GradientTape() as tape:
        state = eng.run(prog, args=make_args(jet)).state
        photons = tf.stack([state.mean_photon(m) for m in range(3)])
        logit   = tf.reduce_sum(photons)
        print("⟨n⟩ =", tf.reduce_sum(photons).numpy())
        loss    = bce(tf.expand_dims(label,0), tf.expand_dims(logit,0))
    vars_ = [tf_s_scale, *tf_DM, *tf_DP, *tf_SM, *tf_SP]  
    grads = tape.gradient(loss, vars_)
    opt.apply_gradients(zip(grads, vars_))
    if step % 5 == 0:
        gnorms = [tf.norm(g).numpy() if g is not None else 0.0 for g in grads]
        print(f"step {step:4d}  loss={loss.numpy():.4f}  "
              f"gnorms={['%.1e'%n for n in gnorms[:6]]}")

# --------- Evaluate and print AUC ---------
def predict_logits(jets_tensor):
    logits = []
    for jet in jets_tensor:
        state = eng.run(prog, args=make_args(jet)).state
        photons = tf.stack([state.mean_photon(m) for m in range(3)])
        logit = tf.reduce_sum(photons).numpy()
        logits.append(logit)
    return np.array(logits)

logits_val = predict_logits(jets_val)
auc_val = roc_auc_score(labels_val.numpy(), logits_val)
print(f"Validation AUC: {auc_val:.4f}")

logits_test = predict_logits(jets_test)
auc_test = roc_auc_score(labels_test.numpy(), logits_test)
print(f"Test AUC: {auc_test:.4f}")

⟨n⟩ = 3.4734027
step    0  loss=0.0305  gnorms=['0.0e+00', '1.5e-03', '2.1e-03', '3.2e-03', '1.3e-03', '1.6e-04']
⟨n⟩ = 2.8781526
⟨n⟩ = 2.2450297
⟨n⟩ = 4.7176275
⟨n⟩ = 4.5668917
⟨n⟩ = 4.9504476
step    5  loss=0.0071  gnorms=['0.0e+00', '2.8e-03', '1.2e-03', '1.4e-03', '3.0e-04', '3.3e-05']
⟨n⟩ = 1.5067389
⟨n⟩ = 2.715891
⟨n⟩ = 2.6605167
⟨n⟩ = 4.8122396
⟨n⟩ = 2.9025989
step   10  loss=2.9560  gnorms=['0.0e+00', '2.0e-02', '3.3e-01', '3.1e-01', '8.3e-02', '2.0e-02']
⟨n⟩ = 1.6540494
⟨n⟩ = 3.1371894
⟨n⟩ = 4.7245255
⟨n⟩ = 4.2510567
⟨n⟩ = 3.467289
step   15  loss=0.0307  gnorms=['0.0e+00', '3.4e-03', '6.6e-03', '8.4e-03', '5.7e-03', '8.7e-04']
⟨n⟩ = 2.4578595
⟨n⟩ = 2.16289
⟨n⟩ = 1.7647659
⟨n⟩ = 2.0694866
⟨n⟩ = 2.2685585
step   20  loss=2.3670  gnorms=['0.0e+00', '1.2e-01', '1.1e-01', '4.3e-02', '2.2e-01', '2.0e-03']
⟨n⟩ = 2.3735878
⟨n⟩ = 2.0818286
⟨n⟩ = 2.6800995
⟨n⟩ = 3.220101
⟨n⟩ = 3.0935752
step   25  loss=3.1379  gnorms=['0.0e+00', '2.0e-01', '9.7e-02', '7.9e-02', '4.5e-01', '1.0e-02']
⟨

KeyboardInterrupt: 

: 